In [1]:
D,U = eig(rand(5,5))

(Complex{Float64}[2.4185638948461006 + 0.0im,0.3182993521571254 + 0.0im,-0.4431823887563463 + 0.0im,-0.2874035930523595 + 0.48623271313745886im,-0.2874035930523595 - 0.48623271313745886im],
5x5 Array{Complex{Float64},2}:
  0.55242+0.0im  0.0740851+0.0im   0.709601+0.0im  …    -0.173063-0.0598999im
 0.387012+0.0im  -0.239474+0.0im  -0.323109+0.0im       -0.146587+0.463534im 
 0.486102+0.0im   0.657809+0.0im  -0.466429+0.0im     -0.00519235-0.234094im 
 0.259241+0.0im  -0.678311+0.0im   0.183075+0.0im       -0.251326-0.144586im 
  0.49148+0.0im   0.210583+0.0im  -0.375493+0.0im        0.768904-0.0im      )

In [3]:
U

5x5 Array{Complex{Float64},2}:
  0.55242+0.0im  0.0740851+0.0im   0.709601+0.0im  …    -0.173063-0.0598999im
 0.387012+0.0im  -0.239474+0.0im  -0.323109+0.0im       -0.146587+0.463534im 
 0.486102+0.0im   0.657809+0.0im  -0.466429+0.0im     -0.00519235-0.234094im 
 0.259241+0.0im  -0.678311+0.0im   0.183075+0.0im       -0.251326-0.144586im 
  0.49148+0.0im   0.210583+0.0im  -0.375493+0.0im        0.768904-0.0im      

## Stop storing intermediate quadratics

The only value of an intermediate quadratic is in preparing for the next intermediate. As long as the rotation matrix is stored, all else can be thrown away. This should reduce memory allocation requirements.

In [1]:
include("../src/TemporalInstanton.jl")
using TemporalInstanton

In [2]:
inputData = load_rts96_data(return_as_type=true)

# Thermal model parameters:
inputData.Tamb = 35.0 # C
inputData.T0 = 60.0 #46.0 # initial line steady-state temp

inputData.time_values = 0.0:30.0:300.0 # five minutes in 30-sec steps
inputData.int_length = 300. # seconds = 5 min

Gp,Dp,Rp = inputData.G0,inputData.D0,inputData.R0
inputData.G0 = [0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp]
inputData.D0 = [0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp]
inputData.R0 = [Rp;1.1*Rp;1.2*Rp;1.3*Rp;1.4*Rp;1.5*Rp]

@time results = solve_temporal_instanton(inputData);
n = 73
nr = 18
T = 6
score,x,θ,α,diffs,xopt = process_instanton_results(results,
    n,
    nr,
    T);

finished 10/104
finished 20/104


finished 30/104


finished 40/104
finished 50/104
finished 60/104
finished 70/104
finished 80/104
finished 90/104
finished 100/104
finished 110/104
 11.506727 seconds (9.22 M allocations: 2.044 GB, 2.51% gc time)


In [4]:
sort(score)

104-element Array{Float64,1}:
    0.263788
    0.327316
    1.28861 
    2.20415 
    2.32297 
    2.42316 
    4.67876 
    4.86524 
    4.89917 
    5.67827 
    5.93862 
    6.43917 
    6.46089 
    ⋮       
  178.086   
  303.003   
  370.614   
  370.614   
  391.307   
  428.944   
  923.98    
  979.199   
 1319.85    
 1390.03    
 1967.27    
 1967.27    

The score vector is exactly the same, but total allocation seems unchanged as well. Nevertheless, each QCQP solution should require less RAM overall because every intermediate step overwrites the results of the last one.

## Stop computing a full eigendecomposition unnecessarily

When we go to diagonalize the quadratic constraint (after kenel mapping), we only need to modify a small subset of the variables. I have been taking a full eigendecomposition of the $\mathbf{N}$ matrix rather than focusing on the part that corresponds to auxiliary angle variables. Perhaps I cannot change this step, but turning a 1k-dimensional eigendecomposition into a 6-dimensional one is a huge upside.

When we use the nullspace basis matrix $\mathbf{N}$ to map the problem onto the kernel of $\mathbf{A}$, we can view the transformation in the following way:

$$
\begin{bmatrix} \vec{y}_1 \\ \vec{y}_2 \\ \vec{y}_3 \end{bmatrix} = \begin{bmatrix} \mathbf{N}_1
  \\ \mathbf{N}_2 \\ \mathbf{N}_3 \end{bmatrix} \vec{x}
$$

In [3]:
?eigs

search: 

```rst
::
           eigs(A, [B,]; nev=6, which="LM", tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)

Computes eigenvalues ``d`` of ``A`` using Lanczos or Arnoldi iterations for
real symmetric or general nonsymmetric matrices respectively. If ``B`` is
provided, the generalized eigenproblem is solved.

The following keyword arguments are supported:
 * ``nev``: Number of eigenvalues
 * ``ncv``: Number of Krylov vectors used in the computation; should satisfy
    ``nev+1 <= ncv <= n`` for real symmetric problems and ``nev+2 <= ncv <= n``
    for other problems, where ``n`` is the size of the input matrix ``A``.
    The default is ``ncv = max(20,2*nev+1)``.
    Note that these restrictions limit the input matrix ``A`` to be of
    dimension at least 2.
 * ``which``: type of eigenvalues to compute. See the note below.

   ========= ======================================================================================================================
   ``which`` type of eigenvalues
   --------- ----------------------------------------------------------------------------------------------------------------------
   ``:LM``   eigenvalues of largest magnitude (default)
   ``:SM``   eigenvalues of smallest magnitude
   ``:LR``   eigenvalues of largest real part
   ``:SR``   eigenvalues of smallest real part
   ``:LI``   eigenvalues of largest imaginary part (nonsymmetric or complex ``A`` only)
   ``:SI``   eigenvalues of smallest imaginary part (nonsymmetric or complex ``A`` only)
   ``:BE``   compute half of the eigenvalues from each end of the spectrum, biased in favor of the high end. (real symmetric ``A`` only)
   ========= ======================================================================================================================

 * ``tol``: tolerance (:math:`tol \le 0.0` defaults to ``DLAMCH('EPS')``)
 * ``maxiter``: Maximum number of iterations (default = 300)
 * ``sigma``: Specifies the level shift used in inverse iteration. If ``nothing`` (default), defaults to ordinary (forward) iterations. Otherwise, find eigenvalues close to ``sigma`` using shift and invert iterations.
 * ``ritzvec``: Returns the Ritz vectors ``v`` (eigenvectors) if ``true``
 * ``v0``: starting vector from which to start the iterations

``eigs`` returns the ``nev`` requested eigenvalues in ``d``, the corresponding Ritz vectors ``v`` (only if ``ritzvec=true``), the number of converged eigenvalues ``nconv``, the number of iterations ``niter`` and the number of matrix vector multiplications ``nmult``, as well as the final residual vector ``resid``.

.. note:: The ``sigma`` and ``which`` keywords interact: the description of eigenvalues searched for by ``which`` do _not_ necessarily refer to the eigenvalues of ``A``, but rather the linear operator constructed by the specification of the iteration mode implied by ``sigma``.

   =============== ================================== ==================================
   ``sigma``       iteration mode                     ``which`` refers to eigenvalues of
   --------------- ---------------------------------- ----------------------------------
   ``nothing``     ordinary (forward)                 :math:`A`
   real or complex inverse with level shift ``sigma`` :math:`(A - \sigma I )^{-1}`
   =============== ================================== ==================================
```

eigs eigvecs eigvals eigvals! leading_ones leading_zeros

